In [807]:
import pandas as pd
import matplotlib.pylab as plt
import numpy as np
from mpltools import style
from mpltools import layout
import seaborn as sns
style.use('ggplot')
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0)
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
from sklearn.grid_search import GridSearchCV
from sklearn import cross_validation
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import auc
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from pandas_ply import install_ply, X, sym_call
install_ply(pd)
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction import DictVectorizer
from sklearn import naive_bayes
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sknn import mlp

In [808]:
def encode_onehot(df, cols):
    """
    One-hot encoding is applied to columns specified in a pandas DataFrame.
    
    Modified from: https://gist.github.com/kljensen/5452382
    
    Details:
    
    http://en.wikipedia.org/wiki/One-hot
    http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html
    
    @param df pandas DataFrame
    @param cols a list of columns to encode
    @return a DataFrame with one-hot encoding
    """
    vec = DictVectorizer()
    
    vec_data = pd.DataFrame(vec.fit_transform(df[cols].to_dict(outtype='records')).toarray())
    vec_data.columns = vec.get_feature_names()
    vec_data.index = df.index
    
    df = df.drop(cols, axis=1)
    df = pd.concat([df, vec_data], axis=1)
    return df

In [809]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
daily = pd.read_csv("daily_aggregate.csv")
roaming = pd.read_csv("roaming_monthly.csv")
calendar = pd.read_csv("calendar_ref.csv")
contract = pd.read_csv("contract_ref.csv")

In [810]:
scores = ['precision', 'recall', 'f-score', 'auc']
def get_cross_validation_scores(clf, X, y, n_folds=10, labels=['M', 'R']):
    cross_validation_scores = np.zeros((len(labels) + 1, len(scores), n_folds))
    i = 0
    for train, test in StratifiedKFold(y, n_folds=n_folds):
        clf = clf.fit(X[train], y[train])
        y_pred = clf.predict(X[test])
        sc = precision_recall_fscore_support(y[test], y_pred, labels=labels)
        tsc = precision_recall_fscore_support(y[test], y_pred)
        for s in range(len(scores)):
            for l in range(len(labels) + 1):
                if (l == len(labels)):
                    continue
                cross_validation_scores[l][s][i] = sc[s][l]
        cross_validation_scores[len(labels)][0][i] = precision_score(y[test], y_pred)
        cross_validation_scores[len(labels)][1][i] = recall_score(y[test], y_pred)
        cross_validation_scores[len(labels)][2][i] = f1_score(y[test], y_pred)
        cross_validation_scores[len(labels)][3][i] = auc(y[test], y_pred, reorder=True)
        print f1_score(y[test], y_pred)
        i += 1
        
    return cross_validation_scores
def mean(x): return "%.5f" % (sum(x) / len(x))
def cross_validation_report(cv, labels):
    row_format ="{:>15}" * (len(scores) + 1)
    print row_format.format("", *scores)
    for l in range(len(labels)):
        print row_format.format(labels[l], *map(mean, cv[l]))
    cv_tr = np.zeros((len(scores)))
    for s in range(len(scores)):
        cv_tr[s] = 0.0
        for l in range(len(labels)): cv_tr[s] += float(map(mean, cv[l])[s])
    cv_tr = map(lambda x: x / len(labels), cv_tr)
    print row_format.format("total/av", *cv_tr)

In [811]:
def convert_gender(g):
        return {"Female": "F", "f": "F", "F": "F", "Male": "M", "m": "M", "M":"M",
                "Not Entered":"Unknown","Unknown":"Unknown"}[g]
contract["GENDER"] = contract["GENDER"].apply(convert_gender)
contract["VALUE_SEGMENT"][pd.isnull(contract["VALUE_SEGMENT"])] = "Unknown"
contract["HANDSET_NAME"][pd.isnull(contract["HANDSET_NAME"])] = "Other"
contract["AGE"][contract["AGE"] < 10] = 99
#contract["APPLE"] = contract["HANDSET_NAME"].apply(lambda x: x.lower().count("apple") > 0)
#contract = contract.drop(["HANDSET_NAME"], axis=1)
def convert_segment(s):
    return {"Core": 1, "Med-Low": 2, "Med-High": 3, "High":4,
           "Premium": 5, "Platinum": 6, "Unknown": 0}[s]
contract["VALUE_SEGMENT"] = contract["VALUE_SEGMENT"].apply(convert_segment)
contract = encode_onehot(contract, ["GENDER"])
contract["AGE_UNKNOWN"] = contract["AGE"] == 99
contract = encode_onehot(contract, ["RATE_PLAN"])
contract["APPLE"] = contract["HANDSET_NAME"].apply(lambda x: x.lower().count("apple") > 0)
contract["GALAXY"] = contract["HANDSET_NAME"].apply(lambda x: x.lower().count("galaxy") > 0)
contract = contract.drop(["HANDSET_NAME"], axis=1)


/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:18: FutureWarning: the 'outtype' keyword is deprecated, use 'orient' instead


In [812]:

def feature_engineer(ds):
   
    global contract
    #ds["USAGE_MEAN"] = (ds["206_USAGE"] + ds["207_USAGE"] \
    #+ ds["208_USAGE"] + ds["209_USAGE"] + ds["210_USAGE"])  / 5.0
    cols = ["206_USAGE", "207_USAGE", "208_USAGE", "209_USAGE", "210_USAGE"]
    s_cols = ["206_SESSION_COUNT", "207_SESSION_COUNT", "208_SESSION_COUNT", "209_SESSION_COUNT", "210_SESSION_COUNT"]
    ds["OVERALL_DIFFERENCE"] = ds[cols[4]] - ds[cols[0]]
    dif_cols = ["207-206", "207-208", "208-209", "209-210"]
    s_dif_cols = ["207-206", "207-208", "208-209", "209-210"]
    ds["210_EXCEEDS"] = ds["210_USAGE"] > 1000
    ds["209_EXCEEDS_SESSIONS"] = ds["209_USAGE"] * 1.0 / ds["209_SESSION_COUNT"] > 3.5
   
    for i in range(len(dif_cols)):
        ds[dif_cols[i]] = ds[cols[i + 1]] - ds[cols[i]]
        #ds[s_dif_cols[i]] = ds[s_cols[i + 1]] - ds[s_cols[i]]
     
    ds["OVERALL_CHANGE"] = ds["207-206"] + ds["207-208"] + ds["208-209"] + ds["209-210"] 
    res = ds.merge(contract, on=["CONTRACT_KEY"])
    # Till we know how to handle
    
    res["211_USAGE"] = (res["210_USAGE"] - res["206_USAGE"])* (5/4.0) + res["206_USAGE"] 
    res["MEAN"] = 0.0
    for m in cols: res["MEAN"] += res[m]
    res["MEAN"] /= 5.0
    res["MEAN"] = res["211_USAGE"] > res["MEAN"] + 500
    res["DIFFERENCE_SIG"] = res["209-210"] > res["207-206"] + 200
    

#     session_usages = [s.split("_")[0] + "_SESSION_USAGE" for s in cols]

#     for i in range(len(s_cols)):
#         res[session_usages[i]] = res[cols[i]] * 1.0 / res[s_cols[i]]
    
#     res["SU_MEAN"] = 0
#     for m in session_usages: res["SU_MEAN"] += res[m]
#     res["SU_MEAN"] /= 5.0
#     res["211_SESSION_USAGE"] = (res["210_SESSION_USAGE"] -
#                                 res["206_SESSION_USAGE"])* (5/4.0) + res["206_SESSION_USAGE"]
#     res["SU_MEAN"] = res["211_SESSION_USAGE"] > res["SU_MEAN"] + 3
    
    
    res = res.drop(cols, axis=1)
#     res = res.drop(session_usages, axis=1)
    res = res.drop(s_cols, axis=1)
    
    return res

In [813]:
train_ready = feature_engineer(train)
test_ready = feature_engineer(test)

In [814]:
q = train_ready
#grid = GridSearchCV(estimator=model, param_grid=dict(n_estimators=ns))
X = q[(q.keys().difference(['TARGET', "CONTRACT_KEY"]))].values
y = q["TARGET"].values
#y.shape = (len(y), 1)

In [815]:
labels = [0, 1]
cv = get_cross_validation_scores(naive_bayes.GaussianNB(), X, y, labels=labels)
cross_validation_report(cv, labels)

0.451445515196
0.452972159518
0.478540772532
0.518188567186
0.405169628433
0.418035426731
0.435643564356
0.422170594092
0.421509686039
0.430908458058
                     precision         recall        f-score            auc
              0        0.85778        0.87489        0.86604     6024.50000
              1        0.46873        0.42459        0.44346     1517.70000
       total/av       0.663255        0.64974        0.65475         3771.1


In [739]:
model = naive_bayes.GaussianNB()
model.fit(X, y)
X_test = test_ready[test_ready.keys().difference(["CONTRACT_KEY"])].values
X_test.shape
test_ready["TARGET"] = model.predict(X_test)

In [740]:
res = test_ready[["CONTRACT_KEY", "TARGET"]]
res.columns = ["CONTRACT_KEY", "PREDICTED_TARGET"]
res.to_csv("submission.csv", index=False)

In [256]:
clf = mlp.Classifier(
        layers=[
            mlp.Layer('Linear', units=202),
            mlp.Layer('Rectifier', units=190),
            mlp.Layer('Softmax')],
        learning_rate=0.000001,
        n_stable=10,
        n_iter=10,
        verbose=False)

In [318]:
grid_sgd = [
  {'alpha': [0.1, 0.01, 0.001, 0.0001, 0.000001], 'average': [True, False]},
  {'class_weight': [{0:0.8, 1:0.2}], 'loss': ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
  'penalty': ['none', 'l2', 'l1', 'elasticnet']}
]
grid_forest = [
    {'n_estimators': [5, 10, 20, 30, 50, 100, 200]}
]


clf = GridSearchCV(AdaBoostClassifier(), grid_forest , cv=5)
clf.fit(X, y)


KeyboardInterrupt: 

In [305]:
clf.best_params_

{'n_estimators': 200}